## Car Damage Severity Classification

This notebook implements yolov11 classification to classify car damage severity into minor, moderate, and severe. The dataset is made by Prajwal Bhamere and comes from https://www.kaggle.com/datasets/prajwalbhamere/car-damage-severity-dataset/data.

### Dependencies

In [2]:
import torch
import torchvision.transforms as T

from ultralytics import YOLO
from ultralytics.data.dataset import ClassificationDataset
from ultralytics.models.yolo.classify import ClassificationTrainer, ClassificationValidator

### Base YOLO model

Now let's load the base YOLOv11 Classification checkpoint for training

In [8]:
model = YOLO("yolo11n-cls.pt", verbose=True)

### Data Preprocessing

Now let's implement a classification dataset class with our own augmentations. These are 

In [4]:
class CarDamageDataset(ClassificationDataset):
    """A customized dataset class for image classification with enhanced data augmentation transforms."""

    def __init__(self, root: str, args, augment: bool = False, prefix: str = ""):
        """Initialize a customized classification dataset with enhanced data augmentation transforms."""
        super().__init__(root, args, augment, prefix)

        train_transforms = T.Compose(
            [
                T.Resize((args.imgsz, args.imgsz)),
                T.RandomHorizontalFlip(p=args.fliplr),
                T.RandomVerticalFlip(p=args.flipud),
                T.RandAugment(interpolation=T.InterpolationMode.BILINEAR),
                T.ColorJitter(brightness=args.hsv_v, contrast=args.hsv_v, saturation=args.hsv_s, hue=args.hsv_h),
                T.ToTensor(),
                T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
                T.RandomErasing(p=args.erasing, inplace=True),
            ]
        )

        val_transforms = T.Compose(
            [
                T.Resize((args.imgsz, args.imgsz)),
                T.ToTensor(),
                T.Normalize(mean=torch.tensor(0), std=torch.tensor(1)),
            ]
        )
        self.torch_transforms = train_transforms if augment else val_transforms


class CustomizedTrainer(ClassificationTrainer):
    """A customized trainer class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train", batch=None):
        """Build a customized dataset for classification training and the validation during training."""
        return CarDamageDataset(root=img_path, args=self.args, augment=mode == "train", prefix=mode)


class CustomizedValidator(ClassificationValidator):
    """A customized validator class for YOLO classification models with enhanced dataset handling."""

    def build_dataset(self, img_path: str, mode: str = "train"):
        """Build a customized dataset for classification standalone validation."""
        return CarDamageDataset(root=img_path, args=self.args, augment=mode == "train", prefix=self.args.split)

In [ ]:
model.train(data="../dataset", trainer=CustomizedTrainer, epochs=100, patience=20, imgsz=224, batch=64)
# model.train(data="../dataset", epochs=100, patience=20, imgsz=224, batch=64)

New https://pypi.org/project/ultralytics/8.3.231 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.228  Python-3.12.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../dataset, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opse

: 

In [6]:
model.val(data="../dataset", validator=CustomizedValidator, imgsz=224, batch=64)


Ultralytics 8.3.228  Python-3.12.7 torch-2.9.1+cu128 CUDA:0 (NVIDIA GeForce RTX 4080 SUPER, 16376MiB)
YOLO11n-cls summary (fused): 47 layers, 1,529,867 parameters, 0 gradients, 3.2 GFLOPs
train: C:\git\classwork\csc_2611\projects\car_damage_classification\dataset\train... found 1383 images in 3 classes  
val: C:\git\classwork\csc_2611\projects\car_damage_classification\dataset\val... found 248 images in 3 classes  
test: None...
val: Fast image access  (ping: 0.00.0 ms, read: 105.225.1 MB/s, size: 7.3 KB)
val: Scanning C:\git\classwork\csc_2611\projects\car_damage_classification\dataset\val... 248 images, 0 corrupt: 100% ━━━━━━━━━━━━ 248/248  0.0s
               classes   top1_acc   top5_acc: 0% ──────────── 0/4  5.0s


RuntimeError: DataLoader worker (pid(s) 80692, 80720, 80728, 77504, 55528, 67092, 78628, 73964) exited unexpectedly